In [ ]:
#import sys
#!{sys.executable} -m pip install numpy --upgrade
#!{sys.executable} -m pip install pandas --upgrade
#!{sys.executable} -m pip install matplotlib --upgrade
#!{sys.executable} -m pip install sklearn --upgrade


import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import psycopg2

import warnings
warnings.filterwarnings('ignore')



# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))
os.chdir('../PumpDataDiscovery')
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

In [ ]:

def get_all_as_df():
    conn = psycopg2.connect(
    database="postgres", user='ad_postgres', password='AWSw0rksh0p', 
    host='db-anomalydetect-postgres.chanowujpkf4.us-east-1.rds.amazonaws.com')
    GET_ALL_ROWS = 'Select * from waterpump order by timestamp'
    try:
        with conn:
            df = pd.read_sql_query(GET_ALL_ROWS, conn)
            df.set_index('timestamp', inplace=True)
    except (Exception, psycopg2.DatabaseError) as err:
        print(err)
    else:
        return df
    finally:
        conn.close()

In [ ]:
def get_null_list(df):
    """Get list of sums of nulls.
    Get list by columns showing how many nulls are in each column
    :param df: DataFrame
    :type: Pandas DataFrame
    :return: List of sums of nulls
    :type: List
    """
    nulls_series = df.isnull().sum()
    return nulls_series

In [ ]:
def drop_bad_cols(df, col_list):
    """Drop unusable columns inplace

    :param df: DataFrame
    :type: Pandas DataFrame
    :param col_list: List of unusable column names
    :return: none
    """
    df.drop(col_list, axis=1, errors='ignore', inplace=True)

In [ ]:
def machine_status_to_numeric(df):
    """Make 'machine_status" column numeric
    Numeric values are 0: 'NORMAL';, 1: 'BROKEN', 0.5: 'RECOVERING'
    :param df: DataFrame
    :type: Pandas DataFrame
    :return: none
    """
    status_values = [(df['machine_status'] == 'NORMAL'), (df['machine_status'] == 'BROKEN'),
                     (df['machine_status'] == 'RECOVERING')]
    numeric_status_values = [0, 1, 0.5]

    df['machine_status'] = np.select(status_values, numeric_status_values, default=0)

In [ ]:
def get_failure_times(df):
    """Get DataFrame of rows where 'machine_status' has a value of 1

    :param df: DataFrame
    :return: Failure times
    :type: DatetimeIndex
    """
    return df[df['machine_status'] == 1].index


In [ ]:
def plot_specific_cols(df):
    df[['sensor_03', 'sensor_04', 'sensor_21', 'machine_status']].plot( kind='line', 
                                             figsize=(10,4),subplots=True)
    plt.tight_layout()
    plt.show()

In [ ]:
def get_failure_times(df):
    """Get DataFrame of rows where 'machine_status' has a value of 1

    :param df: DataFrame
    :return: Failure times
    :type: DatetimeIndex
    """
    return df[df['machine_status'] == 1].index

In [ ]:
def separate_data(df, failure_times):
    """Slice data by failure times
    If there are 7 failure times, then produce 7 slices as DataFrames

    :param df: All Data
    :type: Pandas DataFrame
    :param failure_times: DatetimeIndex
    :return: DataFrames for training, validation and testing
    :type: tuple of DataFrames
    """
    df_val = df.loc[:(failure_times[0] + pd.Timedelta(seconds=60 * 120)), :]
    df_test = df.loc[(failure_times[0] + pd.Timedelta(seconds=60 * 120)):(
            failure_times[1] + pd.Timedelta(seconds=60 * 120)), :]
    df_train = df.loc[failure_times[1] + pd.Timedelta(seconds=60 * 120):, :]

    return df_train, df_val, df_test

In [ ]:
def replace_nan_with_mean(df):
    """
    First replace all empty cells with NaN
    Then replace NaN columnwise with mean of each column inplace
    :param df: DataFrame
    :type: Pandas DataFrame
    :return: none
    """
    cols = df.columns
    df_no_blank = df.replace(r'^\s*$', np.nan, regex=True)
    df.fillna(value=df_no_blank[cols].mean(), inplace=True)

In [ ]:
def get_scaler(training_df):
    """Get a scaler for training data
    Apply MinManScaler to the training data using the range (0, 1)
    :param training_df:
    :type: Pandas DataFrame
    :param sensor_names: List of sensor names(feature names)
    :type: list
    :return: scaler for the training data
    :type: MinMaxScaler
    """
    sensor_names = training_df.columns[:-2]
    min_max_scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = min_max_scaler.fit(training_df[sensor_names])

    return scaler

In [ ]:
def scale_dataframe(scaler, data_df):
    """Transform given dataframe using given scaler as applied to the given columns

    :param scaler: The scaler that has been fit to the dataframe
    :type: MinMaxScaler
    :param data_df: Dataframe to be scaled
    :type: Pandas DataFrame
    :param sensor_names: List of sensor names
    :type: List of string
    :return: array of scaled data
    :type: ndarray
    """
    sensor_names = data_df.columns[:-2]
    scaled_data = scaler.transform(data_df[sensor_names])
    return scaled_data

In [ ]:
def get_PCA(train_scaled_data, num_components=None):
    """Get PCA that has been fit to the scaled training data
    PCA is the Principal Component Analysis.  Training data features are each ranked by their maximum variance from
    all the other features.
    :param train_scaled_data: scaled training data
    :type: ndarray
    :param num_components: Number of features in the training data
    :return: Fit PCA
    :type: PCA
    """
    pca = PCA(n_components=num_components).fit(train_scaled_data)
    return pca

In [ ]:
def plot_pca(pca, n_features):
    #pca = PCA().fit(data_rescaled)


    plt.rcParams["figure.figsize"] = (12,6)

    fig, ax = plt.subplots()
    xi = np.arange(1, n_features, step=1)
    y = np.cumsum(pca.explained_variance_ratio_)

    plt.ylim(0.0,1.1)
    plt.plot(xi, y, marker='o', linestyle='--', color='b')

    plt.xlabel('Number of Components')
    plt.xticks(np.arange(1, n_features, step=1)) #change from 0-based array index to 1-based human-readable label
    plt.ylabel('Cumulative variance (%)')
    plt.title('The number of components needed to explain variance')

    plt.axhline(y=0.95, color='r', linestyle='-')
    plt.text(25, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

    ax.grid(axis='x')
    plt.show()

In [ ]:
def transform_df_by_pca(pca, df_data, scaled_data, num_pc_to_include):
    """Reduce the number of features in the training data by the parameter num_features_to_include.

    :param pca: The PCA for the training data
    :type: PCA
    :param df_data: data dataframe
    :type: Pandas DataFrame
    :param scaled_data: Array of scaled data
    :type: ndarray
    :param num_pc_to_include: Number of features to include.  Currently, this number is determined
    by the num of PC's that were chosen by the PCA to reach 95% ( sum of explained_variance_ratio)
    :type: int
    
    :return: DataFrame with data that has been scaled and whose dimensions have been reduced.  This DataFrame has
    the same index as the param df_data
    :type: Pandas DataFrame
    """

    data_transformed = pca.transform(scaled_data)  # ndarray
    df_transformed = pd.DataFrame(data_transformed)
    pcs = ['pc' + str(i + 1) for i in range(pca.n_components_)]
    df_transformed.columns = pcs

    df_transformed.index = df_data.index
    df_smaller = df_transformed[pcs[:num_pc_to_include]]
    df_smaller['machine_status'] = df_data['machine_status'].values

    return df_smaller

## Get Data 
Read the data into a dataframe, set the dataframe's index to the 'timestamp' column and parse the dates so that they become datetime objects.  Get a count of null values for each column.

In [ ]:
#df = pd.read_csv('sensor.csv', index_col='timestamp', parse_dates=True)
df = get_all_as_df()



## Null count for each column
Make a list of columns that have too many nulls and remove those columns from the dataframe.

In [ ]:
nulls_series = get_null_list(df)
print(nulls_series)

In [ ]:
bad_col_list = ['Unnamed: 0', 'sensor_00', 'sensor_15', 'sensor_50', 'sensor_51']
drop_bad_cols(df, bad_col_list)

## Convert 'machine_status' values with numerics
Convert the textual machine_status values, 'BROKEN', 'NORMAL', 'RECOVERING' to numeric values

In [ ]:
machine_status_to_numeric(df)
df.head()

## Plot some sensor data
Plot three sensor values and compare the plots to the plot of the machine_status column.

In [ ]:
plot_specific_cols(df)

## Replace blank and NaN values with mean for each column
Replace all blank values in the dataframe with NaN and then replace all NaN values with the mean of each column.

In [ ]:
replace_nan_with_mean(df)

## Get failure times
Get all failure times.  

In [ ]:
failure_times = get_failure_times(df)
print("Failure times: {}".format(failure_times))

## Split data by failure times
Separate data into training, validation and test dataframes.  The slicing will be done as follows:
    
    Validation data will consist of all rows from the start of the data file to two minutes after the first failure time
    Test data will consist of all rows from two minutes after the first failure to two minutes after the second failure time
    Training data will consist of all rows from two minutes after the second failure to the end of the data file.

In [ ]:
df_train, df_val, df_test = separate_data(df, failure_times)

## Create a MinMaxScaler for training data
Fit a MinMaxScaler to the training data

In [ ]:
training_scaler = get_scaler(df_train)

## Transform all data using MinMaxScaler that was fit to training data
Scale transform the training, validation, and test data using the training scaler.

In [ ]:
scaled_train = scale_dataframe(training_scaler, df_train)
scaled_val = scale_dataframe(training_scaler, df_val)
scaled_test = scale_dataframe(training_scaler, df_test)


## Create a PCA that is fit on the training data
Fit the scaled training data with a PCA analysis.  Plot the cumulative sum of the variance to determine how many PC's 
are needed to reach 95% of the threshold.

The Principal Component Analysis (PCA) makes linear combinations ( Called Principal Components) of all the features and then ranks those combinations by their per cent contribution to the total variance. We then see how many Principal
Components we need to account for a threshold of 95% of the total variance.  We will see by the graph generated that we will need 12 PC's to account for 95%.

We will then use 12 PC's instead of 47 sensors to do our model training, testing and prediction.  So by using the PCA transformation, we reduce our need for 47 features to only 12 features.

In [ ]:
pca = get_PCA(scaled_train)
plot_pca(pca, len(df_train.columns) - 1)

## Transform all the data using the fitted PCA
Transform the scaled training, validation and test data using the fitted PCA 

In [ ]:
num_pc_to_include = 12
df_train_pca = transform_df_by_pca(pca, df_train, scaled_train, num_pc_to_include)
df_val_pca = transform_df_by_pca(pca, df_val, scaled_val, num_pc_to_include)
df_test_pca = transform_df_by_pca(pca, df_test, scaled_test, num_pc_to_include)


In [ ]:
df_train_pca.head()